# a quick-start notebook for ChiantiPy

## similar to the quick-start guide in the online documentation

In [ ]:
import os

In [ ]:
os.environ['XUVTOP']

This short tutorial will demonstrate some of the capabilities of ChiantiPy and the CHIANTI database. 
It assumes that you know what the CHIANTI database provides and why you want to use it. 
The current implementation uses Version 0.6.4 of ChiantiPy and version 8.0 of the CHIANTI database and mainly provides access to methods concerned with single ions. An ion such as Fe XIV is specified by the string ‘fe_14’, in the usual CHIANTI notation.

Bring up a Jupyter notebook

    > jupyter notebook

It is easiest to do this in the directory where the notebook (.ipynb) resides.

Once you open QuickStart.ipynb in the browser window, you will see something like this.

In my Ipython 00-setup.py file I have already performed

In [ ]:
import ChiantiPy
import ChiantiPy.core as ch
import ChiantiPy.tools.filters as chfilters
import ChiantiPy.tools.io as chio
import ChiantiPy.tools.data as chdata
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
matplotlib qt

In [ ]:
autoreload 3

In [ ]:
ChiantiPy.__version__

In [ ]:
import IPython
print(' IPython version = %i.%i.%i'%(IPython.version_info[0],IPython.version_info[1],IPython.version_info[2]))

It is useful to open a qtconsole where are the calculations can be easily examined

## if you have ipyparallel installed it is necessary to do the following before opening a qtconsole

In [ ]:
import ipyparallel
ipyparallel.bind_kernel = lambda : None

In [ ]:
qtconsole

In [ ]:
chianti_version = chio.versionRead()

In [ ]:
chianti_version

# setting default values

### ChiantiPy determines a number of default setting on instantiation. Without doing anything, the default values

### listed below will be used,

| **setting**     |**default**                     | **possible values**      |
---------- | --------------------------- | ---------------------|
|wavelength | angstrom                    | angstrom, nm, ev, kev|
|flux       | energy                      | energy, photon       |
|abundfile  | sun_photospheric_2015_scott | any.abund            |
|ioneqfile  | chianti                     | any.ioneq            |

### to use any of the other possible values, check out the notes/setting_default_values in the documentation

the defaults can be checked

In [ ]:
chdata.Defaults.keys()

In [ ]:
chdata.Defaults['wavelength']

## setting some matplot parameters for pretty plotting

In [ ]:
mpl.rcParams['xtick.labelsize'] = 16
mpl.rcParams['ytick.labelsize'] = 16
mpl.rcParams['axes.labelsize'] = 16
mpl.rcParams['axes.linewidth'] = 1.5

Level populations
=================

As a start, we will examine the various properties of the Fe XIV emissivities as a function of temperature and density. So, let’s define a numpy array of temperatures and a value for the electron density

In [ ]:
temp = 10.**(5.8 + 0.05*np.arange(21.))
edens = 1.e+9

In ChiantiPy, temperatures are currently given in degrees Kelvin and densities as the number electron density per cubic cm. Then, construct fe14 as would be typically done

In [ ]:
fe14 = ch.ion('fe_14', temperature=temp, eDensity=edens, em=1.e+27)

alternately, since **temperature** and **eDensity** are the first keyword arguments
fe14 = ch.ion('fe_14', temp, edens)

In [ ]:
fe14.popPlot(addLegend=False)

produces a matplotlib plot window were the population of the top 10 (the default) levels are plotted as a function of temperature.

If the level populations had not already been calculated, popPlot() would have invoked the populate() method which calculates the level populations and stores them in the Population dictionary, with keys = [‘protonDensity’, ‘population’, ‘temperature’, ‘density’].

The populations vs. temperature is not particularly interesting.  Plotting them vs. density is more interesting and will come latere

A ChiantiPy Convention
======================

Classes and function of ChiantiPy start with lower case letters. 
----------------------------------------------------------------

Data attached to the instantiation of a class will start with a capital letter. For example,
--------------------------------------------------------------------------------------------

fe14.populate() creates fe14.Population containing the level population information

fe14.emiss() creates fe14.Emiss containing the line emissivity information

fe14.intensity() creates fe14.Intensity containing the line intensity information that includes the elemental abundance and the ionization equilibrium

fe14.spectrum() creates fe14.Spectrum contain the line and continuum spectrum information

# Attributes of an ion

In [ ]:
fe12 = ch.ion('fe_12', setup = False)

a list of attributes can be found using dir(), but this is kind of messy.  A list is provided below

* AbundAll
* Abundance
* Defaults
* Dielectronic
* FIP
* FileName
* GrndLevels
* HigherName
* Ion
* IonStr
* Ip
* Iso
* Labels
* RStar
* RadTemperature
* Spectroscopic
* Z

In [ ]:
print('nuclear charge:  %i'%(fe12.Z))

In [ ]:
print('ionization potential   %6.2f  eV'%(fe12.Ip))

In [ ]:
print('isoelectronic sequence %i, the number of electrons'%(fe12.Iso))

Spectral Line Intensities
=========================

In [ ]:
fe14.intensity()
for akey in sorted(fe14.Intensity):
    print('%10s'%(akey))

the units for line intensities, the key **intensity**, are erg cm$^{-2}$ s$^{-1}$ sr$^{-1}$ / $\int N_e N_H d\ell $, if the emission meassure **em** *not* unspecified

In [ ]:
fe14.intensityPlot(wvlRange=[210.,220.])

In [ ]:
fe14.intensityPlot(index=11, wvlRange=[210.,220.])

In [ ]:
fe14.intensityPlot(index=10, wvlRange=[210.,220.], relative=True, doTitle=False, lw=2)

In [ ]:
fe14.intensityList(wvlRange=[210.,220.])

here, we have used the default value for the keyword argument **top** which specified how many of the most intense lines to list.  Also, **index** can be specified to give a different temperature, and **relative** can be set to 1 give relative emissivities

In [ ]:
fe14.intensityList(wvlRange=[210.,220.], relative=1, index=11)

optionally, an output file could also be created by setting the keyword **outFile** to the name of the desired name

The effect of electron density in line intensities
----------------------------------------------------

In [ ]:
temp = 2.e+6
dens = 10.**(6. + 0.1*np.arange(61))
fe14 = ch.ion('fe_14', temp, dens)
fe14.popPlot()

a plot of the population of the top 10 levels is produced as a function of the electron density

In [ ]:
fe14.intensityRatio(wvlRange=[210.,220.])

to obtain ratios of lines widely separated in wavelength, the **wvlRanges** keyword can be used

In [ ]:
temp = 10.**(5.8 + 0.05*np.arange(21.))
edens = 1.e+9
fe12 = ch.ion('fe_12', temperature=temp, eDensity=edens)
fe12.intensityRatio(wvlRanges=[[190.,200.],[1240.,1250.]])

the most recently calculated intensity ratio is stored in fe14.IntensityRatio, with the following keys:

In [ ]:
for akey in sorted(fe14.IntensityRatio):
    print('%10s'%(akey))

G(n,T) function or G(T)
=======================

When G(n,T), for specific spectral line, is multiplied by the line of sight **emission measure**, $\int$ n$_e$ n_$H$ d $\ell$, it provides the predicted value of the line intensity in units of erg cm$^{-2}$ s$^{-1}$ sr$^{-1}$, if the value for **flux** in the** Defaults** is set to **energy** (the default value).  If **flux** is set to photon, the intensity is given in units of


photons cm$^{-2}$ s$^{-1}$ sr$^{-1}$.

A **chiantirc** file is included with the ChiantiPy distribution. If it is placed in $HOME/.chianti it will be read when ChiantiPy is initiated.  Editing this file allows you to specify the values of **flux** that you want and other things such as the set of elemental abundances and the ionization equilibrium.

In [ ]:
temp = 10.**(5.8 + 0.05*np.arange(21.))
dens = 1.e+9
fe14 = ch.ion('fe_14', temp, dens)

In [ ]:
fe14.gofnt(wvlRange=[210., 220.],top=3)

this brings up a plot of relative line ratios vs temperature and a single selection widget, similar to the intensityRatio process.  Multiple lines can be selected with the *control* key and their G(T) functions will summed.

The **g(n,T)** calculation is stored in the **Gofnt** dictionary, with keys

In [ ]:
sorted(fe14.Gofnt.keys())

In [ ]:
fe14.Gofnt['wvl']

while this is a fairly straightforward way to get a G(T) function, it is not very practical to use for a more than a handful of lines.  For if the fe_14 line at 211.3172 is in a list of lines to be analyzed, a more practical way is the following

In [ ]:
fe14.intensity()
dist = np.abs(np.asarray(fe14.Intensity['wvl']) - 211.3172)
idx = np.argmin(dist)
print(' wvl = %10.3f '%(fe14.Intensity['wvl'][idx]))

In [ ]:
plt.loglog(temp,fe14.Intensity['intensity'][:,idx], 'k')

once the axes are properly scaled, this produces the same values as fe14.Gofnt['gofnt']

# Ionization Equilibrium

For the Fe XIV example, the temperature was chosen to center around 2.e+6. It was not immediately apparent why this was done but in most of the following examples it is necessary to pick an appropriate temperature. This can be done with the ioneq class. To look at the ionization equilibrium for the iron ions (Z = 26, or ‘fe’)

In [ ]:
fe = ch.ioneq(26)
fe.load()
fe.plot()
plt.tight_layout()

brings up a plot showing the ionization equilibrium for all of the stages of iron as a function of temperature

This is pretty crowded and we are only interested in Fe XIV (fe_14), so

In [ ]:
plt.figure()
fe.plot(stages=[13,14,15],tRange=[1.e+6, 6.e+6], yr = [1.e-2, 0.4])
plt.tight_layout()

produces a plot of the ionization equilibria of Fe XIII, XIV and XV over a limited temperature range (tRange) and vertical range (yr)

## Calculating a New Ionization Equilibrium

The ionization equilibrium file in the CHIANTI distribution (chianti.ioneq) is calculated in the low-density limit for 121 temperatures
between 10$^4$ and 10$^9$ K.  For example, if you are interested in temperatures below 10$^4$ K, it is possible to use the ioneqMake
functions.

In [ ]:
temp = np.logspace(3.5, 5.0, 16)
filename = 'my.ioneq'
directory = os.path.join(os.environ['XUVTOP'], 'ioneq')
reference = ['myself', '2025']
ch.Ioneq.ioneqMake(temperature = temp, filename = filename, directory = directory, reference = reference)

places a new ionization equilibrium file ( my.ioneq ) is placed in the standard CHIANTI directory for ionization equilibrium.
The only required keyword argument is the filename.  If the directory is not specified, the new ioneq file is placed in your home directory.
This calculates the ionization equilibrium for all ions from H through Zn.

the new ioneq file can be plotted as just above:

In [ ]:
plt.figure(layout = 'tight')
carbon = ch.ioneq(6)
carbon.load('my.ioneq')
carbon.plot(tRange = [3.e+3, 1.e+5])
plt.tight_layout()

Intensity Ratios
================

In [ ]:
temp = 10.**(5.8 + 0.05*np.arange(21.))
dens = 1.e+9

In [ ]:
fe14 = ch.ion('fe_14', temperature = temp, eDensity = dens)

In [ ]:
fe14.intensityRatio(wvlRange=[210., 225.])

this brings up a plot showing the relative emissivities on the top 10 Fe XIV lines
followed by a dialog where you can selector the numerator(s) and denominator(s) of the desired intensity ratio
so the specified ratio is then plotted
the intensityRatio as a function of temperature and density can be saved to a text file with the following

In [ ]:
fe14.intensityRatioSave()

In [ ]:
fe14.intensityRatioSave(outFile='myratio.txt')

saves the ratio to the ascii file 'myratio.txt'

or, the intensity ratio as a function of electron density

In [ ]:
temp = 2.e+6
dens = 10.**(6. + 0.1*np.arange(61))
fe14 = ch.ion('fe_14', temp, dens)

In [ ]:
fe14.intensityRatio(wvlRange=[210.,220.], top = 5)

Spectrum of a single ion
========================

In [ ]:
temp = 10.**(5.8 + 0.05*np.arange(21.))
edens = 1.e+9

In [ ]:
fe14 = ch.ion('fe_14', temperature = 2.e+6, eDensity = edens, em=1.e+27)

In [ ]:
wvl = 200. + 0.125*np.arange(801)
fe14.spectrum(wvl)

In [ ]:
plt.figure()
plt.plot(wvl, fe14.Spectrum['intensity'])
xy = plt.axis()
xy

## or with the newer object scheme

In [ ]:
fig, ax = plt.subplots(tight_layout = True)
ax.plot(wvl, fe14.Spectrum['intensity'], lw = 2)
ax.set_ylim(bottom = 0.)

In [ ]:
plt.axis([200., 300., 0., 400.])
plt.xlabel(fe14.Spectrum['xlabel'], fontsize=14)
plt.ylabel(fe14.Spectrum['ylabel'], fontsize=14)
plt.tight_layout()

this calculates the spectrum of fe_14 over the specified wavelength range and filter it with the default filter which is a gaussian (filters.gaussianR) with a ‘resolving power’ of 1000 which gives a gaussian width of wvl/1000.  Other filters available in chianti.filters include a boxcar filter and a gaussian filter where the gaussian width can be specified directly

the units of the vertical axis is erg cm$^{-2}$ s$^{-1}$ sr$^{-1}$ $\mathring A$$^{-1}$ since the value of the emission measure **em** has been specified

In [ ]:
if hasattr(fe14,'Em'):
    print(' Emission Measure = %12.2e'%(fe14.Em[0]))
else:
    print(' the value for the emission measure is unspecified')

In [ ]:
fe14.spectrum(wvl,filter=(chfilters.gaussian,.4))

calculates the spectrum of fe_14 for a gaussian filter with a width of 0.4 Angstroms.  The current value of the spectrum is kept in fe14.Spectrum with the following keys:

In [ ]:
for akey in sorted(fe14.Spectrum.keys()):
    print(' %10s'%(akey))

In [ ]:
if hasattr(fe14,'Em'):
    print(' Emission Measure = %12.2e'%(fe14.Em[0]))
else:
    print(' the value for the emission measure is unspecified')

### Here, the previously value of the emission measure has been used

In [ ]:
plt.figure()
plt.plot(wvl,fe14.Spectrum['intensity'])
plt.xlabel(fe14.Spectrum['xlabel'], fontsize=14)
plt.ylabel(fe14.Spectrum['ylabel'], fontsize=14)

In [ ]:
xy = plt.axis()
xy

In [ ]:
plt.axis([wvl[0], wvl[-1], 0., 250.])
plt.tight_layout()

### As of **ChiantiPy 0.14.0**, the **ion** class inherits the spectrumPlot method.

In [ ]:
wvlRange = [wvl[0], wvl[-1]]
fe14.spectrumPlot(wvlRange=wvlRange)

## also in 0.14.0 is the saveData method

In [ ]:
saveName = 'fe14_save.pkl'

In [ ]:
fe14.saveData(saveName, verbose=True)

the attributes are saved as a dict and saved as a pickle file

In [ ]:
with open(saveName,'rb') as inpt:
    fe14Dict = pickle.load(inpt)

In [ ]:
for akey in fe14Dict:
    print(' key = %s'%(akey))

In [ ]:
for akey in fe14Dict['Spectrum']:
    print(' key = %s'%(akey))

In [ ]:
plt.figure()
plt.plot(fe14Dict['Spectrum']['wavelength'], fe14Dict['Spectrum']['integrated'])

it is possible to work directly with the saved data

## Using emission measures (EM)

Beginning with CHIANTI version 10, a new directory, em, as been added to contain emission measure files.

## the line-of-sight emission measure is give by

$$
  \large{\int \, n_e \, n_H \, dl \, ({cm}^{-5})}
$$

## and the volumetric emission measure is given by

$$
  \large{\int \, n_e \, n_H \, dV \,( cm^{-3})}
$$

### where the integration is over the source region

In [ ]:
emDir = os.path.join(os.environ['XUVTOP'], 'em')

In [ ]:
emList = os.listdir(emDir)

In [ ]:
for idx, emFile in enumerate(emList):
    print('%i  %s'%(idx, emFile))

at this time, there only 2 files available and we can pick the active region file

In [ ]:
arDict = chio.emRead(emList[1])

In [ ]:
arDict.keys()

In [ ]:
arTemp = arDict['temperature']
arDens = arDict['density']
arEm = arDict['em']

In [ ]:
for idx, atemp in enumerate(arTemp):
    print('%i %10.2e %10.2e %10.2e'%(idx, atemp, arDens[idx], arEm[idx]))

In [ ]:
fe14  = ch.ion('fe_14', arTemp, arDens, em=arEm)

In [ ]:
wvl = np.linspace(200., 300., 10001)

In [ ]:
fe14.spectrum(wvl, filter=(chfilters.gaussian, .03))

In [ ]:
fe14.spectrumPlot(wvlRange=[264., 275.], integrated=True, top=5)

In [ ]:
fe14.spectrumPlot(wvlRange=[210., 220.], index=2, top=5)

### with version 0.14.0, there is a new class, redux

### with this class, the saved data can be restored and all of the apprpriated inherited methods are available

In [ ]:
rdx = ch.redux(saveName, verbose=True)

### the following gives the previous plot

In [ ]:
rdx.spectrumPlot(wvlRange=wvlRange)

In ChiantiPy 0.6, the **label** keyword has been added to the ion.spectrum method, and also to the other various spectral classes.  This allows several spectral calculations for different filters to be saved and compared

In [ ]:
temp = 10.**(5.8 + 0.1*np.arange(11.))
dens = 1.e+9
emeas = np.ones(11, np.float64)*1.e+27

In [ ]:
fe14 = ch.ion('fe_14', temp, dens, em=emeas)

In [ ]:
wvl = 200. + 0.125*np.arange(801)
fe14.spectrum(wvl,filter=(chfilters.gaussian,.4),label='.4')
fe14.spectrum(wvl,filter=(chfilters.gaussian,1.),label='1.')

In [ ]:
for akey in sorted(fe14.Spectrum.keys()):
    print(' %10s'%(akey))

In [ ]:
for akey in sorted(fe14.Spectrum['.4'].keys()):
    print(' %10s'%(akey))

In [ ]:
plt.figure()
plt.plot(wvl,fe14.Spectrum['.4']['intensity'][5],label='0.4')
plt.plot(wvl,fe14.Spectrum['1.']['intensity'][5],'-r', label = '1.0')
plt.axis([wvl[0], wvl[-1], 0., 250.])
plt.xlabel(fe14.Spectrum['.4']['xlabel'], fontsize=14)
plt.ylabel(fe14.Spectrum['.4']['ylabel'], fontsize=14)
plt.legend(loc='upper right', fontsize=14)
plt.tight_layout()

Free-free and free-bound continuum
==================================

The module continuum provides the ability to calculate the free-free, free-bound continuum spectrum for a large number of individual ions.  The two-photon continuum is produced only by the hydrogen-like and helium-like ions

In [ ]:
myIon = 'fe_25'

In [ ]:
temperature = [2.e+7, 3.e+7, 6.e+7]
density = 1.e+9
em = [1.e+27, 1.e+27, 1.e+27]
wvl = 0.5 + 0.002*np.arange(4501)

In [ ]:
c = ch.continuum(myIon, temperature = temperature, em=em)
c.freeFree(wvl)
c.freeBound(wvl)
fe25=ch.ion(myIon, temperature, density, em=em)
fe25.twoPhoton(wvl)
total = c.FreeFree['intensity'] + c.FreeBound['intensity'] + fe25.TwoPhoton['intensity']

In [ ]:
itemp = 1
plt.figure()
plt.plot(wvl, c.FreeFree['intensity'][itemp],label='ff')
plt.plot(wvl, c.FreeBound['intensity'][itemp],label='fb')
plt.plot(wvl,fe25.TwoPhoton['intensity'][itemp],label='2 photon')
plt.plot(wvl, total[itemp], 'k', label='total')
plt.xlabel(c.FreeFree['xlabel'], fontsize=14)
plt.ylabel(c.FreeFree['ylabel'], fontsize=14)
plt.legend(loc='upper right', fontsize=14)
plt.title(' %s  T = %10.2e'%(fe25.IonStr, temperature[itemp]), fontsize=14)
plt.ylim(bottom=0.)
plt.xlim([0., wvl[-1]])
plt.tight_layout()

In [ ]:
myIon = 'o_8'

In [ ]:
temperature = [3.e+6, 6.e+6]
density = 1.e+9
em = [2.e+27,1.e+27]
wvl = 2. + 0.2*np.arange(701)

In [ ]:
c = ch.continuum('o_8', temperature = temperature, em=em)
c.freeFree(wvl)
c.freeBound(wvl)
o8 = ch.ion(myIon, temperature, density, em=em)
o8.twoPhoton(wvl)
total = c.FreeFree['intensity'] + c.FreeBound['intensity'] + o8.TwoPhoton['intensity']

In [ ]:
itemp = 1
plt.figure()
plt.plot(wvl, c.FreeFree['intensity'][itemp], label='ff')
plt.plot(wvl, c.FreeBound['intensity'][itemp], label='fb')
plt.plot(wvl, o8.TwoPhoton['intensity'][itemp], label='2 photon')
plt.plot(wvl, total[itemp], 'k', label='total')
plt.ylim(bottom=0.)
#plt.xlim(left = 0., right= 100.)
plt.xlabel(c.FreeFree['xlabel'], fontsize=14)
plt.ylabel(c.FreeFree['ylabel'], fontsize=14)
plt.title(' %s  T = %10.2e'%(o8.IonStr, temperature[itemp]), fontsize=14)
plt.legend(loc='upper right', fontsize=14)
plt.tight_layout()

In [ ]:
itemp = 0
plt.figure()
plt.semilogy(wvl, c.FreeFree['intensity'][itemp],label='ff')
plt.semilogy(wvl, c.FreeBound['intensity'][itemp],label='fb')
plt.semilogy(wvl,o8.TwoPhoton['intensity'][itemp],label='2 photon')
plt.semilogy(wvl, total[itemp], 'k', label='total')
plt.ylim(bottom=1.e-4)
#plt.xlim(left = 0., right= 100.)
plt.xlabel(c.FreeFree['xlabel'], fontsize=14)
plt.ylabel(c.FreeFree['ylabel'], fontsize=14)
plt.title(' %s  T = %10.2e'%(o8.IonStr, temperature[itemp]), fontsize=14)
plt.legend(loc='upper right', fontsize=14)
plt.tight_layout()

In the continuum calculations, Fe XXV in this case, is the target ion for the free-free calculation. For the free-bound calculation, specified ion is also the target ion. In this case, the radiative recombination spectrum of Fe XXV recombining to form Fe XXIV is returned.  **Note** this will bring up an error message.  Things are probably OK for the present.

In [ ]:
for akey in  ChiantiPy.tools.data.Defaults.keys():
   print(' %10s - %s'%(akey,ChiantiPy.tools.data.Defaults[akey]))

The multi-ion class bunch
=========================

The multi-ion class **bunch** [new in v0.6] inherits a number of the same methods inherited by the ion class, for example *intensityList*, *intensityRatio*, and *intensityRatioSave*. As a short demonstration of its usefulness, Widing and Feldman (1989, ApJ, 344, 1046) used line ratios of Mg VI and Ne VI as diagnostics of elemental abundance variations in the solar atmosphere. For that to be accurate, it is necessary that the lines of the two ions have the same temperature response.

In [ ]:
temp = 10.**(5.0+0.1*np.arange(11))
bnch=ch.bunch(temp, 1.e+9, wvlRange=[300.,500.], ionList=['ne_6','mg_6'], abundance='unity', em=1.e+27)

In [ ]:
bnch.intensityRatio(wvlRange=[395.,405.],top=6)

### there seems to be a significant temperature dependence to the ratio, even though both are formed near 4.e+5 K.

## the intensityPlot method can also be used with bunch

In [ ]:
bnch.intensityPlot(wvlRange=[398., 404.], index=5, top=7)

### with version 0.13.0 it is possible to save multi-ion calculations as a pickle file

In [ ]:
dataName = 'mybunch.pkl'
bnch.saveData(dataName, verbose=True)

the saveData method creates a dict of all of the attributes of the bnch instance. The pickle file can be loaded an it is possible to work directly with the data.

In [ ]:
with open(dataName, 'rb') as inpt:
    mybnch = pickle.load(inpt)

In [ ]:
mybnch.keys()

In [ ]:
mybnch['Intensity']['intensity'].shape

In [ ]:
rebnch = ch.redux(dataName, verbose=False)

In [ ]:
rebnch.intensityPlot(index=5, wvlRange=[398., 404.])

with verbose set to True, a list of attributes that are saved is printed

produces the previous figure

### A new keyword argument **keepIons** has been added in v0.6 to the bunch and the 3 spectrum classes.

In [ ]:
temp = 10.**(5.0+0.2*np.arange(6))
dens = 1.e+9

In [ ]:
dwvl = 0.01
nwvl = (406. - 394.)/dwvl
wvl = 394. + dwvl*np.arange(nwvl + 1)
wvlRange = [wvl.min(), wvl.max()]

In [ ]:
bnch2=ch.bunch(temp, dens, wvlRange=wvlRange, elementList=['ne','mg'], keepIons=True, em=1.e+27)

it is also possible to create a spectrum with specified line width.

In [ ]:
bnch2.convolve(wvl,filter=(chfilters.gaussian, 5.*dwvl))

In [ ]:
for one in sorted(bnch2.IonInstances.keys()):
    print('%s'%(one))

In [ ]:
bnch2.spectrumPlot(integrated=True, doLabel=False)
plt.plot(wvl,bnch2.IonInstances['mg_6'].Spectrum['integrated'], 'r', label='mg_6')
plt.xlim(left=398., right=404.)
plt.legend(loc='upper left', fontsize=14)

In [ ]:
bnch2.spectrumPlot(top=7)

Spectra of multiple ions and continuum
======================================

The spectrum for a selection of all of the ions in the CHIANTI database can also be calculated. There are 3 spectral classes.

1.  spectrum - the single processor implementation that can be used anywhere
2.  mspectrum - uses the Python multiprocessing class and **cannot** be used in a IPython qtconsole or notebook
3.  ipymspectrum [new in v0.6] - uses the IPython parallel class and can be used in a IPython qtconsole or notebook


spectrum, mspectrum and ipymspectrum can all be instantiated with the same arguments and keyword arguments.  Most of the examples below use the ipymspectrum class for speed.

As of version 0.13.0, it is now possible to save the calculations with the saveData methods, demonstrated with the bunch class above

The single processor spectrum class,  this may take a while
----------------------------------------------------

In [ ]:
temp = [1.e+6, 2.e+6]
dens = 1.e+9
wvl = 200. + 0.05*np.arange(2001)
emeasure = [1.e+27 ,1.e+27]

In [ ]:
s = ch.spectrum(temp, dens, wvl, filter = (chfilters.gaussian,.2), em = emeasure, doContinuum = False, minAbund=1.e-5)

In [ ]:
plt.figure()
plt.subplot(311)
plt.plot(wvl, s.Spectrum['integrated'])
plt.ylim(bottom=0.)
plt.title('integrated')
plt.subplot(312)
plt.plot(wvl, s.Spectrum['intensity'][0])
plt.ylim(bottom=0.)
plt.subplot(313)
plt.plot(wvl, s.Spectrum['intensity'][1])
plt.ylim(bottom=0.)

### using the matplotlib object scheme

In [ ]:
fig, ax = plt.subplots(ncols = 1, nrows = 3, tight_layout = True)
ax[0].plot(wvl, s.Spectrum['integrated'], lw = 2)
ax[0].set_ylim(bottom = 0.)
ax[0].set_title('Integrated')
ax[1].plot(wvl, s.Spectrum['intensity'][0], lw = 2)
ax[1].set_ylim(bottom = 0.)
ax[2].plot(wvl, s.Spectrum['intensity'][1], lw = 2)
ax[2].set_ylim(bottom = 0.)

### some adjustments are probably needed to get a proper figure

### save the calculations

In [ ]:
saveName = 'spectrum.pkl'

In [ ]:
s.saveData(saveName, verbose=True)

In [ ]:
s.spectrumPlot(integrated=True)

## one can return to the saved data and reload it with the redux class

In [ ]:
rdx = ch.redux(saveName)

In [ ]:
rdx.spectrumPlot(index=1)

The multiple processor mspectrum class
-----------------------------------------------------------

### the class uses the Python multiprocessing module

In [ ]:
temp = [1.e+7, 2.e+7, 3.e+7]
dens = 1.e+9
wvl = np.linspace(1.5, 4., 10001)
emeasure = 1.e+27

In [ ]:
dwvl = wvl[1] - wvl[0]
' dwvl:  %8.4f'%(dwvl)

In [ ]:
core = 4

In [ ]:
sm = ch.mspectrum(temp, dens, wvl, filter = (chfilters.gaussian,5.*dwvl), em = emeasure, doContinuum=False,
                  minAbund=1.e-5, proc=core)

In [ ]:
sm.spectrumPlot(wvlRange=[1.84, 1.88], integrated=True)

### save the calculations

In [ ]:
saveName = 'mspectrum.pkl'

In [ ]:
sm.saveData(saveName, verbose=True)

In [ ]:
sm.spectrumPlot(wvlRange=[1.84, 1.88], index=2)

## another example

## Using differential emission measures (DEM)

Beginning with CHIANTI version 14, the io.demRead function has been added
to read dem file in the existing XUVTOP/dem directory

In [ ]:
demDir = os.path.join(os.environ['XUVTOP'], 'dem')

In [ ]:
demList = os.listdir(demDir)

In [ ]:
for idx, demFile in enumerate(demList):
    print('%i  %s'%(idx, demFile))

select the desired file by index

In [ ]:
flDict = chio.demRead(demList[3])

In [ ]:
flDict.keys()

since we will be looking at X-ray wavelengths, select only the highest temperatures

In [ ]:
flTemp = flDict['temperature'][20:]
flDens = flDict['density'][20:]
flEm = flDict['em'][20:]

In [ ]:
for idx, atemp in enumerate(flTemp):
    print('%3i %10.2e %10.2e %10.2e'%(idx, atemp, flDens[idx], flEm[idx]))

In [ ]:
flDict['ref']

In [ ]:
wvl = 1. + 0.002*np.arange(4501)

In [ ]:
core = 4

In [ ]:
s3 = ch.mspectrum(flTemp, flDens, wvl, filter = (chfilters.gaussian,.015),doContinuum=1, em=flEm, minAbund=1.e-5,
                  verbose=0, proc=core)

### save the calculations

In [ ]:
saveName = 'mspectrum3_dem.pkl'

In [ ]:
s3.saveData(saveName, verbose=True)

In [ ]:
plt.figure()
plt.plot(wvl, s3.Spectrum['intensity'].sum(axis=0))
plt.xlabel(s3.Spectrum['xlabel'], fontsize=14)
plt.ylabel(s3.Spectrum['ylabel'], fontsize=14)
plt.ylim(bottom = 0.)
plt.xlim([0., wvl[-1]])
plt.tight_layout()

In [ ]:
s3.spectrumPlot(top=6, integrated=True)

with doContinuum=1, the continuum can be plotted separately - the default value for doContinuum = True/1

In [ ]:
plt.figure()
plt.plot(wvl, s3.FreeFree['intensity'].sum(axis=0), label='FF')
plt.plot(wvl, s3.FreeBound['intensity'].sum(axis=0), label='FB')
plt.plot(wvl, s3.TwoPhoton['intensity'].sum(axis=0), label='2 Photon')
plt.plot(wvl, s3.Continuum['intensity'].sum(axis=0), 'k', label='Total')
plt.xlabel(s3.Spectrum['xlabel'], fontsize=14)
plt.ylabel(s3.Spectrum['ylabel'], fontsize=14)
plt.ylim(bottom = 0.)
plt.xlim([0., wvl[-1]])
plt.legend(loc='center right', fontsize=14)
plt.tight_layout()

In [ ]:
s3.spectrumPlot(wvlRange=[4., 9.], top=6, integrated=True)

### with the redux class, the save calculations can be restored

In [ ]:
s3r = ch.redux(saveName, verbose=True)

### the redux class inherits the intensityPlot and spectrumPlot methods as well as a few others

In [ ]:
s3r.spectrumPlot(wvlRange=[6., 7.], integrated=True, top=5)

The multiple processor ipymspectrum class
-----------------------------------------

next, we will use the ipymspectrum class.  First, it is necessary to start up the cluster.  In some shell

> ipcluster start --n=4

this will start 4 engines if you have 4 cores.  It will only start an many engines as there are cores available

In [ ]:
temp = [1.e+6, 2.e+6]
dens = 1.e+9
wvl = 200. + 0.05*np.arange(2001)
emeasure = [1.e+27 ,1.e+27]

the following code may raise some RuntimeWarnings like

IPython could not determine IPs ..., or 

Controller appears to be listening on localhost, but not on this machine ...

but the code does actually run

In [ ]:
s = ch.ipymspectrum(temp, dens, wvl, filter = (chfilters.gaussian,.2), em = emeasure, doContinuum=0,
                    minAbund=1.e-5, verbose=True)

In [ ]:
plt.figure()
plt.plot(wvl, s.Spectrum['integrated'])
plt.ylim(bottom=0.)
plt.xlim([wvl[0], wvl[-1]])
plt.title('Integrated')
plt.xlabel(s.Spectrum['xlabel'], fontsize=14)
plt.ylabel(s.Spectrum['ylabel'], fontsize=14)
plt.tight_layout()

Using a 4 core processor running at about 3.5 GHz:
* For a minAbund of 1.e-4, the spectra of 25 ions were calculated in 68 s. On a single processor it took 118 s.
* For a minAbund of 1.e-5, then the spectra of 100 ions would be calculated in 72 s. The previous example with the single processor spectrum class took about  129 s on the same computer.
* For a minAbund of 1.e-6, then the spectra of 170 ions would be calculated in 90 s.  On a single processor it took 183 s.

Using only 3 cores:
* For a minAbund of 1.e-5, then the spectra of 100 ions would be calculated in 73 s.

Using only 2 cores:
* For a minAbund of 1.e-5, then the spectra of 100 ions would be calculated in 88 s.

Using only 1 core, with ch.spectrum:
* For a minAbund of 1.e-5, then the spectra of 100 ions would be calculated in 133 s.

[New in version 0.6] One can also use a different abundance file than the default by specifying the abundanceName keyword. For example, abundanceName = ‘cosmic_1973_allen.abund’. If the specified file is not found in XUVTOP/abundance, then a widget will pop up and one can select the abundance file from a list.

It is also possible to specify a selection of ions by means of the ionList keyword, for example, ionList=[‘fe_11’,’fe_12’,’fe_13’] or with the elementList keyword, for example, elementList=[‘mg’,’si’]


In [ ]:
s2 = ch.ipymspectrum(temp, dens, wvl, filter = (chfilters.gaussian,.2), em = emeasure,
                     doContinuum=0, keepIons=1, elementList=['si'])

Because **keepIons** has been set, the ion instances of all of the ions are maintained in the s2.IonInstances dictionary. It has been possible to compare the spectrum of all of the ions with the spectrum of a single ion.

In [ ]:
fig, [ax1, ax2] = plt.subplots(2,1)
ax1.plot(wvl,s2.Spectrum['intensity'][0])
ax1.set_ylim(bottom=0.)
ax1.set_xlim([wvl[0], wvl[-1]])
ax1.set_ylabel(r'erg cm$^{-2}$ s$^{-1}$ sr$^{-1} \AA^{-1}$', fontsize=14)
ax2.plot(wvl,s2.IonInstances['si_9'].Spectrum['intensity'][0])
ax2.set_ylim(bottom=0.)
ax1.set_xlim([wvl[0], wvl[-1]])
ax2.set_ylabel(r'erg cm$^{-2}$ s$^{-1}$ sr$^{-1} \AA^{-1}$', fontsize=14)
ax2.set_xlabel(r'Wavelength ($\AA$)', fontsize=14)
ax2.set_title('Si IX', fontsize=14)
fig.tight_layout()

In [ ]:
temp = 1.e+7
dens = 1.e+9
wvl = 10. + 0.005*np.arange(2001)

In [ ]:
s0 = ch.ipymspectrum(temp, dens, wvl, filter = (chfilters.gaussian,.015), elementList=['fe'], em=1.e+27)

In [ ]:
s0.spectrumPlot(wvlRange=[12., 18.])

In [ ]:
temp=2.e+7
dens=1.e+9
wvl = 1. + 0.002*np.arange(4501)
s = ch.ipymspectrum(temp, dens, wvl, filter = (chfilters.gaussian,.015),doContinuum=1,
                    em=1.e+27,minAbund=1.e-5,verbose=0)
plt.plot(wvl, s3.Spectrum['intensity'])

In [ ]:
temp=5.e+6
dens=1.e+9
dwvl = .005
nwvl = (50. - 1.)/dwvl
wvl=1. + dwvl*np.arange(nwvl+1)

by setting abundanceName to 1, a widget with a list of possible abundance sets will pop up and allow you to select one.  Alternative, you can set abundanceName to the name of one of the abundance file (without the .abund suffix) and it will use that set.

In [ ]:
s3 = ch.ipymspectrum(temp, dens, wvl, filter = (chfilters.gaussian,5.*dwvl),doContinuum=1, em=1.e+40,minAbund=1.e-5)
plt.figure()
plt.plot(wvl, s3.Spectrum['intensity'])

In [ ]:
print(' number of ions calculated %i '%(len(s3.IonsCalculated)))

In [ ]:
s3.spectrumPlot()

Since the continuum was calculated, it can be plotted separately

In [ ]:
plt.figure()
plt.plot(wvl, s3.FreeFree['intensity'],label='FF')
plt.plot(wvl, s3.FreeBound['intensity'],label = 'FB')
plt.plot(wvl, s3.TwoPhoton['intensity'], label = 'TP')
plt.plot(wvl,s3.FreeBound['intensity']+s3.FreeFree['intensity']+s3.TwoPhoton['intensity'],label='FF+FB+TP')
plt.legend(loc='upper right')

In [ ]:
temperature = 2.e+7
density = 1.e+9
em = 1.e+27
wvl = 1.84 + 0.0001*np.arange(601)
s4 = ch.ipymspectrum(temperature, density ,wvl, filter = (chfilters.gaussian,.0003), em=em, minAbund=1.e-5,verbose=0)

In [ ]:
s4.spectrumPlot(wvlRange=[1.845, 1.88])

# Radiative loss rate

In [ ]:
temp = 10.**(4.+0.05*np.arange(81))
dens = 1.e+9
rl = ch.radLoss(temp, dens, elementList=['h', 'he'])

In [ ]:
plt.figure()
rl.radLossPlot()

### save the data

In [ ]:
saveName = 'rl_h_he.pkl'

In [ ]:
rl.saveData(saveName)

### with version 0.15.0, the class mradLoss is available for doing a multiprocessor calculation
### of the radiation loss

In [ ]:
temp = 10.**(4.+0.05*np.arange(81))
dens = 1.e+9

the following will calculate the radiation loss for elements with an abundance greater the 1.e-5 that of the hydrogen abundance.  In this case the default abundance file is for photospheric abundances

In [ ]:
mrl = ch.mradLoss(temp, dens, minAbund = 1.e-5)

## save the calculations

In [ ]:
saveNamePhot = 'rl_phot_1m5.pkl'

In [ ]:
mrl.saveData(saveNamePhot)

In [ ]:
plt.figure()
mrl.radLossPlot()

### the radiative losses are stored in the dictionary rl.RadLoss

In [ ]:
for akey in sorted(mrl.RadLoss.keys()):
    print(' %s '%(akey))

### it is also possible to set the keyword argument **abundancedName** to use different sets of abundances

In [ ]:
abundDir = os.path.join(os.environ['XUVTOP'], 'abundance')

In [ ]:
abundList = os.listdir(abundDir)

In [ ]:
for idx, aname in enumerate(abundList):
    print('%5i  %s'%(idx, aname))

### to select coronal abundances

In [ ]:
cdx = 2

In [ ]:
myAbund = abundList[cdx]

In [ ]:
mrl2 = ch.mradLoss(temp, dens, abundance=myAbund, minAbund=1.e-5, proc = 4, verbose=1)

In [ ]:
saveNameCoronal = 'rl_coronal_1m5.pkl'

In [ ]:
mrl2.saveData(saveNameCoronal)

In [ ]:
plt.figure()
mrl2.radLossPlot()

### this took 1550s on a relatively slow 6 core processor

In [ ]:
plt.figure()
plt.loglog(temp, mrl2.RadLoss['rate'], 'k', label='Total')
plt.loglog(temp, mrl2.BoundBoundLoss, label = 'BB')
plt.loglog(temp, mrl2.FreeFreeLoss, label = 'FF')
plt.loglog(temp, mrl2.FreeBoundLoss, label = 'FB')
plt.loglog(temp, mrl2.TwoPhotonLoss, label = '2P')
plt.xlabel(mrl2.RadLoss['xlabel'], fontsize=14)
plt.ylabel(mrl2.RadLoss['ylabel'], fontsize=14)
plt.legend(loc='lower center', fontsize=14)
plt.tight_layout()

### to compare photospheric and coronal radiation losses

In [ ]:
rlph = ch.redux(saveNamePhot)

In [ ]:
rlco = ch.redux(saveNameCoronal)

In [ ]:
plt.figure()
plt.loglog(temp, rlph.RadLoss['rate'], label='Phot')
plt.loglog(temp, rlco.RadLoss['rate'], label='Coronal')
plt.ylim(bottom=1.e-23, top=2.e-21)
plt.xlabel(rlph.RadLoss['xlabel'], fontsize=14)
plt.ylabel(rlph.RadLoss['ylabel'], fontsize=14)
plt.legend(loc='lower center', fontsize=14)
plt.tight_layout()